# Machine Learning - CatBoost

In [2]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [3]:
required_libs = [ ("numpy", "numpy"),
                 ("pandas", "pandas"),
                 ("seaborn", "seaborn"),
                 ("matplotlib", "matplotlib"),
                 ("catboost", "catboost"),
                 ("sklearn", "sklearn"),
                 ("ipywidgets", "ipywidgets"),
                 ("shap", "shap"),
                 ("colorama", "colorama"),
                 ("emoji", "emoji")
                ]

In [4]:
def is_lib_exists(name):
    import importlib
    lib = importlib.util.find_spec(name)
    return lib is not None

In [5]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
catboost exists
sklearn exists
ipywidgets exists
shap exists
colorama exists
emoji exists


In [6]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import traceback
import catboost
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
from catboost import *
from catboost import datasets
np.set_printoptions(precision=4)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
%matplotlib inline
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [9]:
from IPython.display import display

In [10]:
# check catboost version
print(catboost.__version__)
!python --version

0.24
Python 3.6.9


In [11]:
# colab setup
if COLAB:
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

Cloning into 'dl-projects'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 2138 (delta 99), reused 32 (delta 14), pack-reused 1977
Receiving objects: 100% (2138/2138), 79.29 MiB | 32.48 MiB/s, done.
Resolving deltas: 100% (1324/1324), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
total 4948M
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr.ft
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr_orig.pkl
-rw------- 1 root root   17M Mar  1 05:47 feature_matrix_2020_feb.ft
-rw------- 1 root root   17M Mar  1 05:39 feature_matrix_2020_jan.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul_orig.pkl
-rw------- 1 root root   17M Jul  5 07:30 feature_matrix_2020_jun.ft
-rw------- 1 

## EDA

In [12]:
from preprocess import *
from utils import feature_selection, plot_feature_importances
from plot import plot_correlation_matrix, plot_labeled_scatter

In [13]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [14]:
jan_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jan.ft")
feb_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_feb.ft")
mar_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_mar.ft")
apr_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_apr.ft")
may_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_may.ft")
jun_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jun.ft")
jul_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jul.ft")

In [15]:
new_data = data.append(jan_2020[data.columns],ignore_index=True)
new_data = new_data.append(feb_2020[data.columns],ignore_index=True)
new_data = new_data.append(mar_2020[data.columns],ignore_index=True)
new_data = new_data.append(apr_2020[data.columns],ignore_index=True)
new_data = new_data.append(may_2020[data.columns],ignore_index=True)
new_data = new_data.append(jun_2020[data.columns],ignore_index=True)
new_data = new_data.append(jul_2020[data.columns],ignore_index=True)
data.shape, new_data.shape 

((959893, 217), (1029893, 217))

In [16]:
data = new_data

In [17]:
data.isna().sum().sort_values(ascending=False)

CUM_SUM(SKEW(Results.TotalStrike))           7685
CUM_MEAN(TREND(Results.DrawNo, DrawDate))    7685
TREND(Results.CUM_SUM(DrawNo), DrawDate)     7685
TREND(Results.CUM_SUM(LuckyNo), DrawDate)    7685
CUM_SUM(SKEW(Results.LuckyNo))               7685
                                             ... 
CUM_SUM(MIN(Results.DrawNo))                    0
NUM_UNIQUE(Results.DAY(DrawDate))               0
NUM_UNIQUE(Results.MONTH(DrawDate))             0
SUM(Results.PERCENTILE(LuckyNo))                0
NumberId                                        0
Length: 217, dtype: int64

In [18]:
data.info(max_cols=500, null_counts=True)
#data.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029893 entries, 0 to 1029892
Data columns (total 217 columns):
 #   Column                                                  Non-Null Count    Dtype         
---  ------                                                  --------------    -----         
 0   NumberId                                                1029893 non-null  int64         
 1   time                                                    1029893 non-null  datetime64[ns]
 2   STD(Results.DrawNo)                                     1028948 non-null  float64       
 3   STD(Results.TotalStrike)                                1028948 non-null  float64       
 4   STD(Results.LuckyNo)                                    1028948 non-null  float64       
 5   MAX(Results.DrawNo)                                     1029893 non-null  int64         
 6   MAX(Results.TotalStrike)                                1029893 non-null  int64         
 7   MAX(Results.LuckyNo)               

In [19]:
feature_matrix = data
feature_matrix = data.fillna(0)

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2

#columns = ['Number', 'time', 'TotalStrike',  'month', 'year', 'Label']
#cols = ['Number', 'time', 'TotalStrike',  'month', 'year', 'Label']
#feature_matrix = feature_selection(data)
def select_features():
  X = feature_matrix.drop(columns=['year', 'time', 'Label', 'TotalStrike'])
  y = feature_matrix.Label
  fs = SelectKBest(score_func=f_classif, k=30)
  X_selected = fs.fit_transform(X, y)
  mask = fs.get_support()
  return X.columns[mask]

In [ ]:
selected_features = select_features()

In [1]:
features_ = selected_features.to_list() + ['NumberId', 'month']
print(features)

NameError: ignored

In [29]:
def split_data():
  pass